In [ ]:
# import pydrake
# import underactuated
# except ImportError:
#     !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
#     from jupyter_setup import setup_underactuated
#     setup_underactuated()

# Setup matplotlib backend (to notebook, if possible, or inline).  
from jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()

In [ ]:
# general imports
import os
import numpy as np
import scipy as sp

# plotting/animation imports
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
rcParams['figure.figsize'] = (8, 5)
from IPython.display import HTML, display

# pydrake imports
from pydrake.all import (VectorSystem, DiagramBuilder, SymbolicVectorSystem, LogOutput, PlanarSceneGraphVisualizer,
                         Variable, SceneGraph, Parser, MultibodyPlant, AddMultibodyPlantSceneGraph, Simulator, sqrt, cos, sin, tan, abs, plot_system_graphviz )
from pydrake.symbolic import if_then_else, logical_and, logical_or
from pydrake.common.containers import EqualToDict


# Simulation Parameters

All parameters defined here. Things like map dimensions, current field, and boat/object properties should go here. Might convert some of these to classes down the road.

In [ ]:
# simulation parameters
map_name = "downstream"                 # map setting
map_width = 500                         # map width [m]
map_height = 500                        # map height [m]
dx = dy = 50                             # discretization step used to plot current vector field
duration = 150                          # timesteps
animate = True                        # whether or not to animate

# initialization parameters
start = [50, 50, np.pi, 0, 0, 0]        # initial state [x, y, theta, x_dot, y_dot, theta_dot]
target = [100, 100, 0, 0, 0, 0]             # goal state [x', y', theta', x_dot', y_dot', theta_dot']

# system constants
m = 4000                                # mass of the boat [kg]
l = 4.0                                 # half length of the boat [m]
w = 5.0                                 # width of the boat [m]
h = 2.0                                 # heigh of boat under water [m]
rho_w = 997.0                           # density of water [kg/m^3]
C = 1.0                                 # water drag coefficient [unitless]
d = 1.5                               # distance of center of drag to CM [m]

C_front = 0.2                           # drag coeff of boat moving forward/back
C_side = 1.15                           # drag coeff of boat moving sideways

# Dynamics

In [ ]:
def drag_coeff_from_angle(angle, C_side, C_front):
  '''
    Takes a weighted average of the side drag coefficient and front/back drag
    coefficient. The weighting corresponds to angle = boat_heading for the
    y-direction coefficients, with the x-direction just being a 90 deg shift

    Currently assumes boat is a symmetrical about both axis 
  '''
  # X is the same as Y shifted by 90 degrees
  return C_front*cos(angle-np.pi/2)**2 + C_side*sin(angle-np.pi/2)**2, C_front*cos(angle)**2 + C_side*sin(angle)**2

def drag_area_from_angle(angle, A_side, A_front):
  '''
    Takes weighted average of boat areas across front and side using same
    convention as drag_coeff_from_angle
  '''
  return A_front*cos(angle-np.pi/2)**2 + A_side*sin(angle-np.pi/2)**2, A_front*cos(angle)**2 + A_side*sin(angle)**2

def get_drag_params(th_h):
  # From paper
  # eps = 0.000001
  # C_front = 0.02058*(l**2*w*h/(sqrt(x_dot**2 + y_dot**2)+eps))**(-1/8)

  A_side = 2*l*h
  A_front = w*h

  # TODO: figure if coefficients are in correct order of magnitude
  scale = 1

  # TODO not sure drag can be vectorized? Is sum of parts equal to whole?
  C_x, C_y = drag_coeff_from_angle(th_h, C_side, C_front)
  A_x, A_y = drag_area_from_angle(th_h, A_side, A_front)

  return C_x*scale, C_y*scale, A_x, A_y

def get_current(x, y, name, vectorize=False):
  '''
    Retrieve x and y current components at a specific coordinate. If vectorize is True,
    then assume x, y inputs are arrays and return a grid of currents along these coordinate
    ranges. Used for plotting the vector field.
  '''  
  currents = {
      "width_varied_river" : 
          [lambda x, y: 0, 
           lambda x, y: ((map_width/2)-abs(x-(map_width/2)))/(map_width/2)*2],
      "downstream" : 
          [lambda x, y: 0, 
           lambda x, y: 1.0],
      "upstream" : 
          [lambda x, y: 0, 
           lambda x, y: -1.0],
      "diagonal_river" : 
          [lambda x, y: 1.0, 
           lambda x, y: 1.0],
      "no_current" :
          [lambda x, y: 0,
           lambda x, y: 0],
      "WHIRLPOOL!" :
          [lambda x, y: x+y,
           lambda x, y: y-x]
  }
  
  if vectorize:
      x_curr = np.zeros((len(x), len(y)))
      y_curr = np.zeros((len(x), len(y)))
      for i in range(len(x)):
          for j in range(len(y)):
              xi = x[i]
              yj = y[j]
              x_curr[i][j] = currents[name][0](xi, yj)
              y_curr[i][j] = currents[name][1](xi, yj)
  else:
      x_curr = currents[name][0](x, y)
      y_curr = currents[name][1](x, y)

  return x_curr, y_curr
  
def generate_dynamics(state, torques):
  # Generate continuous time dynamics: x_dot = f(x, u, t) #

  F_t, th_t = torques
  x, y, th_h, x_dot, y_dot, th_dot = state
  x_dot_c, y_dot_c = get_current(x, y, map_name)

  # Drag coefficient is a function of heading
  C_drag_x, C_drag_y, A_drag_x, A_drag_y = get_drag_params(th_h)

  F_drag_x = 0.5*rho_w*C_drag_x*A_drag_x*abs(x_dot_c - x_dot)*(x_dot_c - x_dot)
  F_drag_y = 0.5*rho_w*C_drag_y*A_drag_y*abs(y_dot_c - y_dot)*(y_dot_c - y_dot)

  # Drag from boat spinning (to stop from spinning infinitely fast)
  T_drag_omega = -rho_w*C_side*h*l**4/4*th_dot*abs(th_dot)

  I = m*((2*l)**2 + w**2) / 12

  # TODO double check, add torque from current
  x_ddot = (F_t*sin(th_h + th_t) + F_drag_x) / m
  y_ddot = (F_t*cos(th_h + th_t) + F_drag_y) / m
  th_ddot = (-F_t*l*sin(th_t) + (F_drag_x*cos(th_h) - F_drag_y*sin(th_h))*d + T_drag_omega)/I

  return [x_dot, y_dot, th_dot, x_ddot, y_ddot, th_ddot]

In [ ]:
# state of the robot (standard cartesian)
x = Variable("x") # horizontal position
y = Variable("y") # vertical position
theta = Variable("theta") # angular position
x_dot = Variable("x_dot") # horizontal velocity
y_dot = Variable("y_dot") # vertical velocity
theta_dot = Variable("theta_dot") # angular velocity
state = [x, y, theta, x_dot, y_dot, theta_dot]

# control torques
thrust = Variable("thrust") # thrust force
thrust_angle = Variable("thrust_angle") # thrust angle
torques = [thrust, thrust_angle]

In [ ]:
def create_boat():
  boat = SymbolicVectorSystem(
              state=state,
              input=torques,
              output=state,
              dynamics=generate_dynamics(state, torques)
          ) 
  return boat

# Plotting

Plotting and animation functions.

In [ ]:
def style_plot(ax, x_label, y_label, mode=None):
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    if mode == 'env':
      ax.set_ylim([0, map_height])
      ax.set_xlim([0, map_width])

def plot_boat(x, y, heading, width, height, ax=None, frame=.1, **kwargs):
    kwargs['edgecolor'] = 'black'
    kwargs['facecolor'] = 'none'
    
    if ax is None:
        ax = plt.gca()

    bottom_left = [x - width/2, y - height/2]
    top_right = [x + width/2, y + height/2]

    boat_shape = Rectangle(bottom_left, width, height, heading, **kwargs)
    boat_heading = None

    ax.add_patch(boat_shape)

    return boat_shape

def static_plot(logger, boat, objects):
    '''
    Render a static plot of the boat, objects, current, along with trajectories traced
    for each of them. Used to initialize the animation, or just quickly plot simulation
    results without taking the time to animate frames.
    '''
    fig1, ax1 = plt.subplots() # plot environment
    fig2, ax2 = plt.subplots() # plot metrics

    # Plot current field
    x_curr = np.arange(0, map_width+dx, dx)
    y_curr = np.arange(0, map_height+dy, dy)
    u, v = get_current(x_curr, y_curr, map_name, vectorize=True)
    ax1.quiver(x_curr, y_curr, u, v, color='teal', width=0.0045)

    # Plot goal
    ax1.plot(target[0], target[1], 'Xg')
  
    # Plot initial objects

    # Highlight start and end of trajectory
    ax1.plot(logger.data()[0,0], logger.data()[1,0], 'og') # start
    ax1.plot(logger.data()[0,-1], logger.data()[1,-1], 'or') # end
    
    # Plot initial boat position
    ax1.plot(logger.data()[0,:], logger.data()[1,:])
    boat_graphic = plot_boat(logger.data()[0,0], logger.data()[1,0], logger.data()[2,0], w, 2*l, ax1)

    # Plot metrics
    ax2.plot(logger.sample_times(), logger.data()[3,:], color='b')
    ax2.plot(logger.sample_times(), logger.data()[4,:], color='k')

    y_min = np.min(np.concatenate((logger.data()[3,:], logger.data()[4,:])))
    y_max = np.max(np.concatenate((logger.data()[3,:], logger.data()[4,:])))
    ax2.set_ylim((y_min-0.5, y_max+0.5))

    ax3 = ax2.twinx()
    ax3.plot(logger.sample_times(), logger.data()[2,:], 'r')
    ax3.plot(logger.sample_times(), logger.data()[5,:], 'r-.')
    ax3.set_ylabel('th_h(t)')

    # Labels and stylistic attributes
    style_plot(ax1, 'x(t)', 'y(t)', 'env')
    style_plot(ax2, 'time', 'xd(t) and yd(t)', 'metrics')

    return fig1, ax1, fig2, ax2, [boat_graphic, list()]

def animate_env(logger, boat, objects):

    fig1, ax1, fig2, ax2, graphics = static_plot(logger, boat, objects)

    def animate(step):

        boat_graphic, objects = graphics
        # move objects
        # move boat
        boat_graphic.set_x(logger.data()[0, step] - w/2)
        boat_graphic.set_y(logger.data()[1, step] - l)

        return [boat_graphic] # list of patches to redraw

    animation = FuncAnimation(fig1, animate, frames=duration*10, interval=1, blit=True)
    display(HTML(animation.to_jshtml())) # displays the animation

# Block Diagram

Block diagram should go here, along with controller.

In [ ]:
def build_block_diagram(boat, controller):

    builder = DiagramBuilder()
    boat = builder.AddSystem(boat)
    boat.set_name('boat')

    # logger to track trajectories
    logger = LogOutput(boat.get_output_port(0), builder)
    logger.set_name('logger')

    # expose boats input as an input port
    builder.ExportInput(boat.get_input_port(0))

    # build diagram
    diagram = builder.Build()
    diagram.set_name('diagram')

    return diagram

# Simulation

Simulation code to animate everything.

In [ ]:
def simulate(initial_state, const_input, boat, controller, duration):

    diagram = build_block_diagram(boat, controller)

    # set up a simulator
    simulator = Simulator(diagram)
    simulator_context = simulator.get_mutable_context()

    # set the initial conditions
    boat = diagram.GetSubsystemByName('boat')
    boat_context = diagram.GetMutableSubsystemContext(boat, simulator_context)
    boat_context.get_mutable_continuous_state_vector().SetFromVector(initial_state)

    # Set Inputs to Boat
    simulator_context.FixInputPort(0, const_input)

    # simulate from time zero to duration
    simulator.AdvanceTo(duration)

    return diagram.GetSubsystemByName('logger')


# Main execution loop

In [ ]:

boat = create_boat()
logger = simulate(start, [0, 0], boat, None, duration)

if animate:
    animate_env(logger, boat, list())
else:
    static_plot(logger, boat, list())
